In [1]:
import numpy as np
import pandas as pd
import xlwings as xw
from openpyxl import load_workbook
import matplotlib.pyplot as plt
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False

import warnings
warnings.filterwarnings("ignore")

import pymysql
from sqlalchemy import create_engine
import json
import datetime
from datetime import timedelta ,time, datetime
from dateutil.relativedelta import relativedelta
import re
import os
import glob
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean

In [2]:
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# 实例化All_Model类
all_models = All_Model()
week_models = Week_Model()
clean = Data_Clean()

In [3]:
def query(sql,
          host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
          user="ylc",
          password="1O8t5lcJ5aMhwwPEUUjS",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

In [4]:
sql = '''
select tmu.id_card_num, om.order_number, om.create_time  from db_digua_business.t_postlease_receivables_monitoring tprm
left join db_digua_business.t_order om on om.id=tprm.order_id
left join db_digua_business.t_member_user tmu on tmu.id=om.user_id
'''
df = query(sql)

In [11]:
df.loc[:, 'num'] = df.groupby('id_card_num').order_number.transform('count')
df.loc[:, 'min_time'] = df.groupby('id_card_num').create_time.transform('min')
df.loc[:, '是否复租'] = np.where(df.create_time==df.min_time, 0, 1)
df = df[df.num>1][['id_card_num', 'order_number', 'create_time', '是否复租']].sort_values(by=['id_card_num', 'create_time'])
with pd.ExcelWriter('F:/需求/总需求/复租订单.xlsx') as writer:
    df.to_excel(writer, sheet_name='复租订单', index=False)